In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%pylab inline
import matplotlib.image as mpimg
from pymatting import *

#load the image
input_image = cv2.imread('/Users/ssrujanaa/Documents/Fall2020/EE-599/Project/Dataset/train2014/COCO_train2014_000000000081.jpg', 1)
#reversal of GBR to RGB
input_image = input_image[:,:,::-1]
#perform gaussion blur
blur = cv2.GaussianBlur(input_image, (5, 5), 0)
blur = blur.astype(np.float32) / 255.0

#use the model.yml file to perform edge detection (pre-trained)
edgeDetector = cv2.ximgproc.createStructuredEdgeDetection("model.yml")
edges = edgeDetector.detectEdges(blurred_float) * 255.0
cv2.imwrite('edge-raw.jpg', edges)



#Performing filtering operation
def Salt_and_pepper_noise(image):
    count = 0
    lastMedian = image
    median = cv2.medianBlur(image, 3)
    while not np.array_equal(lastMedian, median):
        zeroed = np.invert(np.logical_and(median, image))
        image[zeroed] = 0

        count = count + 1
        if count > 70:
            break
        lastMedian = median
        median = cv2.medianBlur(image, 3)


edges_8u = np.asarray(edges, np.uint8)
Salt_and_pepper_noise(edges_8u)
cv2.imwrite('edge.jpg', edges_8u)



#find the significatn contour
def Contour(image):
    contours, hierarchy = cv2.findContours(image,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    level1Meta = []
    for contourIndex, tupl in enumerate(hierarchy[0]):
        if tupl[3] == -1:
            tupl = np.insert(tupl.copy(), 0, [contourIndex])
            level1Meta.append(tupl)
            
    contoursWithArea = []
    for tupl in level1Meta:
        contourIndex = tupl[0]
        contour = contours[contourIndex]
        area = cv2.contourArea(contour)
        contoursWithArea.append([contour, area, contourIndex])
    contoursWithArea.sort(key=lambda meta: meta[1], reverse=True)
    largestContour = contoursWithArea[0][0]
    return largestContour


contour = findSignificantContour(edges_8u)

# Draw the contour on the original image
contourImg = np.copy(src)
cv2.drawContours(contourImg, [contour], 0, (0, 255, 0), 2, cv2.LINE_AA, maxLevel=1)
cv2.imwrite('contour.jpg', contourImg)




